In [14]:
from google.cloud import aiplatform


def create_custom_job_sample(
    project: str,
    display_name: str,
    container_image_uri: str,
    container_args: list,
    credentials,
    location: str = "europe-central2",
    api_endpoint: str = "europe-central2-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.JobServiceClient(client_options=client_options, credentials=credentials)
    custom_job = {
        "display_name": display_name,
        "job_spec": {
            "worker_pool_specs": [
                {
                    "machine_spec": {
                        "machine_type": "n1-standard-8",
                        "accelerator_type": aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
                        "accelerator_count": 1,
                    },
                    "replica_count": 1,
                    "container_spec": {
                        "image_uri": container_image_uri,
                        "command": [],
                        "args": container_args,
                    },
                }
            ]
        },
    }
    parent = f"projects/{project}/locations/{location}"
    response = client.create_custom_job(parent=parent, custom_job=custom_job)
    print("response:", response)

In [17]:
from google.oauth2 import service_account

BUCKET_NAME = "pix2pix-training-artifacts-1171246"
PROJECT_ID = "sandbox-project-462110"
IMAGE_URI = "milonpabis/train_pix2pix:0.1"

credentials = service_account.Credentials.from_service_account_file("sandbox-project-462110-215ba3072b18.json")
aiplatform.init(project=PROJECT_ID, credentials=credentials)

create_custom_job_sample(
    project=PROJECT_ID,
    credentials=credentials,
    display_name="pix2pix_training-maps",
    container_image_uri=IMAGE_URI,
    container_args= [
            f"--bucket_name={BUCKET_NAME}", 
            "--execution_type=cloud", 
            "--bucket_data_prefix=data_maps",
            "--gpus=all"
    ],
    location="us-central1",
    api_endpoint="us-central1-aiplatform.googleapis.com"
)

ResourceExhausted: 429 The following quota metrics exceed quota limits: aiplatform.googleapis.com/custom_model_training_nvidia_t4_gpus